In [1]:
import dgl
import math
import torch
import numpy as np
from os import path
from pathlib import Path
from dgl.data import DGLDataset
from ToyDGLDataset import ToyDGLDataset, GetNodeFeatureVectors, GetEdgeFeatureVectors, GetNeighborNodes, GetEdgeList

Using backend: pytorch


In [2]:
datasetName = 'ToyDataset01'
datasetDir = path.join('/home/andrew/GNN_Sandbox/GraphDatasets', datasetName)
dataset = ToyDGLDataset(datasetName, datasetDir)

Done loading data from cached files.


In [3]:
dataset.printProperties()

Num Graph classes: 2
Graph classes: [0, 1]
Number of graphs: 4000
Number of all nodes in all graphs: 188523
Number of all edges in all graphs: 10270270
Dim node features: 5
Node feature keys: ['P_t', 'Eta', 'Phi', 'Mass', 'Type']
Dim edge features: 3
Edge feature keys: ['DeltaEta', 'DeltaPhi', 'RapiditySquared']


In [4]:
print(len(dataset))
graph, label = dataset[0]
print(graph)
print(f'Label: {label}')
print(GetNodeFeatureVectors(graph))

4000
Graph(num_nodes=13, num_edges=156,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float64)})
Label: 0
tensor([[ 6.9153e+01, -8.3210e+00,  1.6391e+00,  5.3223e-01,  1.0000e+00],
        [ 5.4858e+01, -8.3092e-01,  5.3403e+00,  7.5453e-02,  1.0000e+00],
        [ 3.1999e+01, -8.3449e+00,  2.1668e+00,  3.5149e-01,  2.0000e+00],
        [ 4.2509e+01, -4.3009e+00,  2.9509e+00,  2.1676e-01,  0.0000e+00],
        [ 2.3563e+01, -5.8565e+00,  1.1033e+00,  7.6687e-01,  1.0000e+00],
        [ 6.8691e+01,  2.4417e+00,  4.5953e+00,  5.9614e-01,  1.0000e+00],
        [ 9.0249e+01, -4.0531e+00,  1.1712e+00,  7.1518e-01,  2.0000e+00],
        [ 1.9054e+01,  5.4614e+00,  5.6575e+00,  5.6662e-01,  1.0000e+00],
        [ 7.7917e+01,  7.5052e+00,  3.2276e+00,  5.2337e-01,  1.0000e+00],
        [ 1.4645e+01,  7.0957e+00,  2.7560e+00,  4.6849e-01,  1.0000e+00],
        [ 1.2005e+01,  8.3119e+00,  1.3066e-01,  2.4928e-01,

In [5]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
splitIndices = dataset.get_split_indices()

train_sampler = SubsetRandomSampler(splitIndices['train'])
test_sampler = SubsetRandomSampler(splitIndices['test'])

train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=32, drop_last=False)
test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=32, drop_last=False)

In [6]:
it = iter(train_dataloader)
batch = next(it)
print(batch)

[Graph(num_nodes=1399, num_edges=75048,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float64)}), tensor([1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
        0, 1, 0, 0, 0, 1, 0, 1])]


In [7]:
batched_graph, labels = batch
print('Number of nodes for each graph element in the batch:', batched_graph.batch_num_nodes())
print('Number of edges for each graph element in the batch:', batched_graph.batch_num_edges())
print(labels)
# Recover the original graph elements from the minibatch
graphs = dgl.unbatch(batched_graph)
print('The original graphs in the minibatch:')
print(graphs)

Number of nodes for each graph element in the batch: tensor([71, 33, 18, 47, 74, 79, 74, 32, 28, 35, 23, 35, 36, 73, 21, 18, 70, 64,
        15, 19, 68, 25, 81, 28, 28, 55, 33, 27, 15, 65, 44, 65])
Number of edges for each graph element in the batch: tensor([4970, 1056,  306, 2162, 5402, 6162, 5402,  992,  756, 1190,  506, 1190,
        1260, 5256,  420,  306, 4830, 4032,  210,  342, 4556,  600, 6480,  756,
         756, 2970, 1056,  702,  210, 4160, 1892, 4160])
tensor([1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0,
        0, 1, 0, 0, 0, 1, 0, 1])
The original graphs in the minibatch:
[Graph(num_nodes=71, num_edges=4970,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float64)}), Graph(num_nodes=33, num_edges=1056,
      ndata_schemes={'feat': Scheme(shape=(5,), dtype=torch.float64)}
      edata_schemes={'feat': Scheme(shape=(3,), dtype=torch.float64)}), Graph(num_nodes=18, num_

In [8]:
from dgl.nn import GraphConv
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        g.ndata['h'] = h
        return dgl.mean_nodes(g, 'h')

In [9]:
# Create the model with given dimensions
model = GCN(dataset.dim_nfeats, 16, dataset.num_graph_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(50):
    for batched_graph, labels in train_dataloader:
        nodeFeatVec = GetNodeFeatureVectors(batched_graph)
        pred = model(batched_graph, nodeFeatVec)
        loss = F.cross_entropy(pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    #if epoch % 5 == 0:
    print(f'Epoch: {epoch}, Loss: {loss:.3f}')

num_correct = 0
num_tests = 0
for batched_graph, labels in test_dataloader:
    nodeFeatVec = GetNodeFeatureVectors(batched_graph)
    pred = model(batched_graph, nodeFeatVec)
    num_correct += (pred.argmax(1) == labels).sum().item()
    num_tests += len(labels)

print('Test accuracy:', num_correct / num_tests)

Epoch: 0, Loss: 0.428
Epoch: 1, Loss: 0.324
Epoch: 2, Loss: 0.457
Epoch: 3, Loss: 0.217
Epoch: 4, Loss: 0.425
Epoch: 5, Loss: 0.191
Epoch: 6, Loss: 0.185
Epoch: 7, Loss: 0.189
Epoch: 8, Loss: 0.171
Epoch: 9, Loss: 0.269
Epoch: 10, Loss: 0.160
Epoch: 11, Loss: 0.144
Epoch: 12, Loss: 0.117
Epoch: 13, Loss: 0.186
Epoch: 14, Loss: 0.074
Epoch: 15, Loss: 0.201
Epoch: 16, Loss: 0.052
Epoch: 17, Loss: 0.095
Epoch: 18, Loss: 0.112
Epoch: 19, Loss: 0.070
Epoch: 20, Loss: 0.048
Epoch: 21, Loss: 0.246
Epoch: 22, Loss: 0.056
Epoch: 23, Loss: 0.060
Epoch: 24, Loss: 0.079
Epoch: 25, Loss: 0.322
Epoch: 26, Loss: 0.193
Epoch: 27, Loss: 0.057
Epoch: 28, Loss: 0.517
Epoch: 29, Loss: 0.286
Epoch: 30, Loss: 0.050
Epoch: 31, Loss: 0.053
Epoch: 32, Loss: 0.465
Epoch: 33, Loss: 0.079
Epoch: 34, Loss: 0.106
Epoch: 35, Loss: 0.027
Epoch: 36, Loss: 0.243
Epoch: 37, Loss: 0.094
Epoch: 38, Loss: 0.085
Epoch: 39, Loss: 0.140
Epoch: 40, Loss: 0.154
Epoch: 41, Loss: 0.159
Epoch: 42, Loss: 0.305
Epoch: 43, Loss: 0.31

In [11]:
print(graph.nodes[[0,1]])
print(graph.nodes[0])
print(graph.nodes[1])
print(graph.nodes[10])
print(graph.nodes[3])

NodeSpace(data={'feat': tensor([[69.1527, -8.3210,  1.6391,  0.5322,  1.0000],
        [54.8578, -0.8309,  5.3403,  0.0755,  1.0000]], dtype=torch.float64)})
NodeSpace(data={'feat': tensor([[69.1527, -8.3210,  1.6391,  0.5322,  1.0000]], dtype=torch.float64)})
NodeSpace(data={'feat': tensor([[54.8578, -0.8309,  5.3403,  0.0755,  1.0000]], dtype=torch.float64)})
NodeSpace(data={'feat': tensor([[12.0050,  8.3119,  0.1307,  0.2493,  2.0000]], dtype=torch.float64)})
NodeSpace(data={'feat': tensor([[42.5092, -4.3009,  2.9509,  0.2168,  0.0000]], dtype=torch.float64)})


In [12]:
#print(graph.out_edges(7))
#print(graph.edges[(0,1)]) # src, dst
#print(graph.edges[0,1]) # src, dst
print(graph.edges[[0, 0], [1,2]]) # src, dst
print(graph.edges[[10], [3]])
print(graph.edges[[3], [10]])
print(graph.edges[0])
print(graph.edges[1])
#print(graph.edges[0]) # eid

EdgeSpace(data={'feat': tensor([[7.4901e+00, 3.7012e+00, 6.9801e+01],
        [2.3897e-02, 5.2767e-01, 2.7901e-01]], dtype=torch.float64)})
EdgeSpace(data={'feat': tensor([[ 12.6129,   2.8203, 167.0381]], dtype=torch.float64)})
EdgeSpace(data={'feat': tensor([[ 12.6129,   2.8203, 167.0381]], dtype=torch.float64)})
EdgeSpace(data={'feat': tensor([[ 7.4901,  3.7012, 69.8007]], dtype=torch.float64)})
EdgeSpace(data={'feat': tensor([[0.0239, 0.5277, 0.2790]], dtype=torch.float64)})


In [16]:
edgeList = GetNeighborNodes(graph, 0)
nodeFeat = graph.ndata['feat']
print(graph.nodes[0])
print(nodeFeat)
print(edgeList)


NodeSpace(data={'feat': tensor([[69.1527, -8.3210,  1.6391,  0.5322,  1.0000]], dtype=torch.float64)})
tensor([[ 6.9153e+01, -8.3210e+00,  1.6391e+00,  5.3223e-01,  1.0000e+00],
        [ 5.4858e+01, -8.3092e-01,  5.3403e+00,  7.5453e-02,  1.0000e+00],
        [ 3.1999e+01, -8.3449e+00,  2.1668e+00,  3.5149e-01,  2.0000e+00],
        [ 4.2509e+01, -4.3009e+00,  2.9509e+00,  2.1676e-01,  0.0000e+00],
        [ 2.3563e+01, -5.8565e+00,  1.1033e+00,  7.6687e-01,  1.0000e+00],
        [ 6.8691e+01,  2.4417e+00,  4.5953e+00,  5.9614e-01,  1.0000e+00],
        [ 9.0249e+01, -4.0531e+00,  1.1712e+00,  7.1518e-01,  2.0000e+00],
        [ 1.9054e+01,  5.4614e+00,  5.6575e+00,  5.6662e-01,  1.0000e+00],
        [ 7.7917e+01,  7.5052e+00,  3.2276e+00,  5.2337e-01,  1.0000e+00],
        [ 1.4645e+01,  7.0957e+00,  2.7560e+00,  4.6849e-01,  1.0000e+00],
        [ 1.2005e+01,  8.3119e+00,  1.3066e-01,  2.4928e-01,  2.0000e+00],
        [ 6.4847e+01,  6.2803e-02,  1.6592e+00,  2.1720e-03,  2.0000e+00

In [47]:
def ConcatNodeAndEdgeFeatures(graph, nodeFeat, nodeLabel: int):
    neighbors = GetNeighborNodes(graph, nodeLabel)
    nfeat = nodeFeat[nodeLabel]
    efeat = torch.reshape(graph.edges[neighbors].data['feat'], (-1,))
    #print(nfeat)
    #print(efeat)
    return torch.cat((nfeat, efeat))
    
x = ConcatNodeAndEdgeFeatures(graph, nodeFeat, 0)
print(x)

edgeFeatureVec = GetEdgeFeatureVectors(graph)


tensor([ 6.9153e+01, -8.3210e+00,  1.6391e+00,  5.3223e-01,  1.0000e+00,
         7.4901e+00,  3.7012e+00,  6.9801e+01,  2.3897e-02,  5.2767e-01,
         2.7901e-01,  4.0201e+00,  1.3118e+00,  1.7882e+01,  2.4645e+00,
         5.3581e-01,  6.3610e+00,  1.0763e+01,  2.9562e+00,  1.2457e+02,
         4.2679e+00,  4.6786e-01,  1.8434e+01,  1.3782e+01,  4.0184e+00,
         2.0610e+02,  1.5826e+01,  1.5885e+00,  2.5299e+02,  1.5417e+01,
         1.1169e+00,  2.3892e+02,  1.6633e+01,  1.5084e+00,  2.7893e+02,
         8.3838e+00,  2.0081e-02,  7.0289e+01,  4.7891e+00,  4.2202e+00,
         4.0745e+01], dtype=torch.float64)


In [ ]:
print(graph.edata['DeltaPhi'])
print(graph.dstdata['P_t'])

In [45]:
a = torch.tensor(np.arange(5,10))
b = torch.tensor(np.arange(9).reshape((3,3)))

print(a)
print(b)
b = torch.reshape(b, (-1,))
print(b)
print(torch.cat((a,b), dim=0))

tensor([5, 6, 7, 8, 9])
tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8])
tensor([5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8])
